# Examining ProPublica's Facebook Political Ad Data

## Getting the data

The data can be downloaded [here](https://www.propublica.org/datastore/dataset/political-advertisements-from-facebook). You don't have to provide your name, email or organization, but you do have to agree to abide by ProPublica's [Terms of Use](https://www.propublica.org/datastore/terms).

## Filtering to Ads targeted to Missouri

We'll use [pandas](https://pandas.pydata.org/) to read in the .csv file, and filter it to only Missouri-related ads.

In [1]:
import pandas as pd

In [2]:
ads = pd.read_csv("fbpac-ads-en-us.csv")

### How many total ads?

In [3]:
len(ads)

64636

Here's a quick look a the top five rows in the file.

In [4]:
ads.head()

,id,html,political,not_political,title,message,thumbnail,created_at,updated_at,lang,...,targeting,suppressed,targets,advertiser,entities,page,lower_page,targetings,paid_for_by,targetedness
0,23842934264470055,"<div class=""_5pcr userContentWrapper""><div cla...",5,0,Blue Wave 2018,<p>Robert Mueller is an American hero!</p>,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-08-27 18:34:00.351869+00,2018-08-28 13:13:35.255391+00,en-US,...,"<div><div class=""_4-i0 _26c5""><div class=""clea...",f,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",Blue Wave 2018,"[{""entity"": ""Robert Mueller"", ""entity_type"": ""...",https://www.facebook.com/Blue-Wave-2018-222012...,https://www.facebook.com/blue-wave-2018-222012...,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",Blue Wave 2018,3.0
1,23843007348380279,"<div class=""_5pa- userContentWrapper""><div cla...",7,0,Beto O'Rourke,<p>BREAKING: Politico just reported that GOP L...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-08-23 16:17:45.244313+00,2018-09-06 00:11:41.161539+00,en-US,...,"<div><div class=""_4-i0 _26c5""><div class=""clea...",f,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",Beto O'Rourke,"[{""entity"": ""Texas"", ""entity_type"": ""Region""},...",https://www.facebook.com/betoorourke/,https://www.facebook.com/betoorourke/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",Beto for Texas,1.0
2,23842930304260339,"<div class=""_5pcr userContentWrapper""><div cla...",6,0,Phil Bredesen,<p>Too many Tennesseans in rural communities d...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-08-27 18:11:13.179067+00,2018-08-30 01:37:36.504856+00,en-US,...,"<div><div class=""_4-i0 _26c5""><div class=""clea...",f,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",Phil Bredesen,"[{""entity"": ""TVA"", ""entity_type"": ""Organizatio...",https://www.facebook.com/BredesenForSenate/,https://www.facebook.com/bredesenforsenate/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",Bredesen for Senate,3.0
3,6106200884302,"<div class=""_5pcr userContentWrapper""><div cla...",4,1,Emerson College Polling,<p>And so our poll of the week is an <a class=...,https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-08-27 18:35:46.511049+00,2018-08-28 13:13:11.850585+00,en-US,...,"<div><div class=""_4-i0 _26c5""><div class=""clea...",f,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",Emerson College Polling,"[{""entity"": ""Emerson College"", ""entity_type"": ...",https://www.facebook.com/emersonpolling/,https://www.facebook.com/emersonpolling/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",Emerson College Polling,2.0
4,23843007362830528,"<div class=""_5pcr userContentWrapper""><div cla...",5,1,Be A Hero,"<p>Clearly calls, emails, and visits are not w...",https://pp-facebook-ads.s3.amazonaws.com/v/t1....,2018-08-29 17:41:48.659359+00,2018-08-30 11:52:20.125079+00,en-US,...,"<div><div class=""_4-i0 _26c5""><div class=""clea...",f,"[{""target"": ""Age"", ""segment"": ""18 and older""},...",Be A Hero,"[{""entity"": ""U.S."", ""entity_type"": ""Region""}, ...",https://www.facebook.com/BeAHeroTeam/,https://www.facebook.com/beaheroteam/,"{""<div><div class=\""_4-i0 _26c5\""><div class=\...",Be a Hero PAC,1.0


### Filtering by `targets`

Looks's like we want to use the `targets` in our filter criteria. It contains what looks like a JSON string: an array of JSON objects. Let's take a closer look at the distinct values of this column.

In [5]:
ads.targets.value_counts()

[{"target": "Age", "segment": "18 and older"}, {"target": "MinAge", "segment": "18"}, {"target": "Retargeting", "segment": "people who may be similar to their customers"}, {"target": "Region", "segment": "the United States"}]                                                                                                                                                                                                                     6317
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                    5642
[{"target": "Age", "segment": "18 and older"}, {"target": "MinAge", "segment": "18"}, {"target": "Region", "segmen

Each JSON object in each array appears to have two keys: `"target"` and `"segment"`. Just scrolling through the above values, I see objects `"target": "State"` paired with a state name in the `"segment"` value. However, I also see objects with `"target": "Region"` with a state name in the `"segment"` value.

A mapping of distinct `"target"` values to their `"segment"` values would be helpful. Let's build that mapping in the stupid, obvious way.

In [6]:
targets_to_segments = {}

We'll need to parse the json strings in the `targets` columns.

In [7]:
import json

Iterate over each row, then iterate over each target for the row, add the `"target"` value to the `targets_to_segments` dict (if it isn't already there), then add the `"segment"` value to the associated array (if it isn't already there).

In [8]:
for row in ads.iterrows():
    try:
        targets = json.loads(row[1]['targets'])
    except TypeError:
        # Ignore an value we can't parse as json (prob some variant of NULL)
        pass
    for target in targets:
        target_value = target['target']
        if target_value not in targets_to_segments:
            targets_to_segments[target_value] = []
        if 'segment' in target:
            if target['segment'] not in targets_to_segments[target_value]:
                targets_to_segments[target_value].append(target['segment'])

### Targeting options

In [9]:
for k in targets_to_segments:
    print(k)

Age
MinAge
Retargeting
Region
Interest
Like
Activity on the Facebook Family
MaxAge
Gender
List
City
State
Website
Segment
Agency
Language
Engaged with Content


`"Region"`, `"State"` and `"City"` seem most relevant to the purposes of location-based filtering.

### Region segments

In [10]:
for i in sorted(targets_to_segments["Region"]):
    print(i)

Alabama
Alaska
Alberta
Arizona
Arkansas
Australia
Australian Capital Territory
Austria
Belgium
California
Canada
Colorado
Connecticut
Delaware
District of Columbia
England
Florida
France
Georgia
Germany
Hawaii
Idaho
Illinois
Indiana
Indonesia
Iowa
Israel
Kansas
Kentucky
Latvia
Latvija
Louisiana
Maine
Maryland
Massachusetts
Mexico
Michigan
Minnesota
Missouri
Montana
Nebraska
Netherlands
Nevada
New Brunswick
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Ontario
Oregon
Pennsylvania
Poland
Rhode Island
Slovakia
South Carolina
South Dakota
Switzerland
Tennessee
Texas
Thailand
Tokyo
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming
the United Kingdom
the United States
Île-de-France


### State segments

In [11]:
for i in sorted(targets_to_segments["State"]):
    print(i)


Alabama
Alaska
Alberta
Arizona
Arkansas
Bangkok
Basel-City wohnen
Bucks County, Pennsylvania
California
Colorado
Connecticut
Delaware
District of Columbia
Dubai
Florida
Georgia
Hawaii
Hillsborough County, Florida
Idaho
Illinois
Indiana
Iowa
Jalisco
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Brunswick
New Hampshire
New Jersey
New Mexico
New South Wales
New York
North Carolina
North Dakota
Northern District
Ohio
Oklahoma
Ontario
Oregon
Pennsylvania
Phnom Penh
Quebec
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Victoria
Virginia
Washington
West Virginia
Wisconsin
Wyoming
Île-de-France


### City segments

In [12]:
for i in sorted(targets_to_segments["City"]):
    print(i)


Accokeek
Aguanga
Akron
Alameda
Albany
Albuquerque
Aledo
Alexandria
Allegheny Acres
Allentown
Alliance
Amarillo
Anaheim
Anchorage
Ann Arbor
Annetta North
Appleton
Ardmore
Arlington
Asheville
Ashfield
Ashland
Astoria
Athens
Atlanta
Auburn
Auburn Hills
Augusta
Aurora
Austin
Avon Lake
Avondale
Baker City
Ballwin
Baltimore
Bangkok
Bartlett
Beaverton
Belle Fourche
Belle View
Bellefontaine Neighbors
Belleville
Bellevue
Bellingham
Bellport
Belmont
Bemidji
Berkeley
Bernardsville
Bethesda
Bethlehem
Binghamton
Birchwood
Birmingham
Bismarck
Bloomfield
Bloomington
Blue River
Bluffton
Boca Raton
Boerne
Boise
Bolingbrook
Bolton
Bon Air
Boonville
Boston
Bothell
Boulder
Boxborough
Boyden
Bradenton
Bradford
Brentwood
Brevard
Bridgeport
Bridgeton
Bridgewater
Brighton
Bristow
Broken Bow
Brookline
Brooklyn
Brunswick
Buena Park
Buffalo
Buffalo Grove
Burien
Burlingame
Burlington
Burrton
Byron
Camarillo
Cambridge
Campbellsville
Canonsburg
Canyon
Cape Coral
Carmel
Carrboro
Cary
Casselberry
Castle Rock
Castro 

### Does ever ad with a `"City"` target also include `"State"` target?

In [13]:
targets_w_city_wo_state = []
for row in ads.iterrows():
    try:
        targets = json.loads(row[1]['targets'])
    except TypeError:
        # Ignore an value we can't parse as json (prob some variant of NULL)
        pass
    has_city = 'City' in [i['target'] for i in targets]
    has_state = 'State' in [i['target'] for i in targets]
    if has_city and not has_state:
        targets_w_city_wo_state.append(targets)

In [14]:
len(targets_w_city_wo_state)

0

Looks like yes (phew).

## Filtering to ads targeted to Missouri (as a `"State"` or `"Region"`)

Since my pandas skills are rusty (particularly with JSON in the mix), we'll just do this the stupid, obvious way.

Here's a placeholder for all the records we want.

In [15]:
mo_ads = []

Loop over the dataframe, if `targets` includes a `"segment"` value of `"Missouri"`, keep it.

In [16]:
for row in ads.iterrows():
    try:
        targets = json.loads(row[1]['targets'])
    except TypeError:
        # Ignore an value we can't parse as json (prob some variant of NULL)
        pass
    has_mo = False
    for target in targets:
        if 'segment' in target.keys():
            if target['segment'] == 'Missouri':
                has_mo = True
    if has_mo:
        mo_ads.append(row[1])

### How many Missouri ads?

In [17]:
len(mo_ads)

546

### Outputting to a new .csv

Now open a new csv file, and write each Missouri ad to it.

In [18]:
from csv import DictWriter

In [19]:
with open('fbpac-ads-MO.csv', 'w', newline='') as f:
    writer = DictWriter(f, fieldnames=list(ads))
    writer.writeheader()
    for ad in mo_ads:
        writer.writerow(ad.to_dict())

## Double-check

Let's read this new file back in and make sure there aren't any obvious problems.

In [20]:
out_ads = pd.read_csv("fbpac-ads-mo.csv")

In [21]:
len(out_ads)

546

### Check that `id` values are unique

In [25]:
out_ads.id.value_counts()

23842922325240188    1
6083893229461        1
23843009419610078    1
23842777351530276    1
23843020430170078    1
6093655990164        1
6101521429387        1
6084252953484        1
6103275672450        1
6110123039825        1
6082889133958        1
23843029742130097    1
6101667484669        1
6082762828797        1
23842905051200327    1
6092685613799        1
6086171725715        1
6096401233821        1
23842793773750057    1
23842928501490539    1
6084948361115        1
23842860179640091    1
23843029757700078    1
23842668246350404    1
6104954851235        1
23842764633890569    1
6092552200098        1
6088548954028        1
23842744374140678    1
23842749495480239    1
                    ..
23842858515150078    1
23842932896900105    1
23843020619610078    1
6078729561171        1
23842746625450239    1
6092685605599        1
23842865664640239    1
23842728912550140    1
23842974931920594    1
6102978643035        1
23842848462330086    1
23842720688950517    1
23843118283

### Check distinct `targets` values

In [23]:
out_ads.targets.value_counts()

[{"target": "Age", "segment": "18 and older"}, {"target": "MinAge", "segment": "18"}, {"target": "Region", "segment": "Missouri"}, {"target": "Retargeting", "segment": "people who may be similar to their customers"}]                                                                                                               59
[{"target": "Age", "segment": "18 and older"}, {"target": "MinAge", "segment": "18"}, {"target": "Region", "segment": "Missouri"}, {"target": "List"}]                                                                                                                                                                                 50
[{"target": "Age", "segment": "18 and older"}, {"target": "MinAge", "segment": "18"}, {"target": "Region", "segment": "Missouri"}]                                                                                                                                                                                                     32
[{"target"